In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import Autoformer
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)

In [5]:
n_inputs = [5,10,15,21,42,84]
hidden_size = [60,90,120,150,180]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_head=[2,3,4,5,6]
decoder_input_size_multiplier=[0.3,0.4,0.5,0.6,0.7]
dropout=[0,0.2,0.3,0.4]
factor=[2,3,4]
conv_hidden_size=[12,22,32,42,52]
decoder_layers = [1,2,3]

# Test

In [6]:
test_length=int(len(df['NKE'])*0.3)
df1=df[int(len(df['NKE'])*0.7*0.5):].unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

# H=1

In [7]:
model = Autoformer(h=1, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
              random_seed=289990)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
forecasts = forecasts.dropna()
if "Autoformer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Autoformer"].values
else:
  Y_hat=forecasts["Autoformer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 289990


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.0038823748858443224 | QLIKE: 0.031327789176844194 | MAE: 0.0023991619381682427 | MAPE: 0.17929995607058705


In [8]:
if "Autoformer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Autoformer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

In [9]:
df2.to_excel("Autoformer forecasts.xlsx")

# H=5

In [10]:
H=5

model = Autoformer(h=H, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
              random_seed=289990)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "Autoformer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Autoformer"].values
else:
  Y_hat=forecasts["Autoformer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "Autoformer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Autoformer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("Autoformer forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 289990


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.004685890392810515 | QLIKE: 0.045655637620412866 | MAE: 0.0028616063222255315 | MAPE: 0.2163525510402775


# H=10

In [11]:
H=10

model = Autoformer(h=H, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
              random_seed=289990)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "Autoformer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Autoformer"].values
else:
  Y_hat=forecasts["Autoformer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "Autoformer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Autoformer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("Autoformer forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 289990


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.00569438683319411 | QLIKE: 0.06058975524508375 | MAE: 0.0033390929853410484 | MAPE: 0.24237385792826155
